# Imports

In [45]:
import random, sys, string, os
from collections import Counter
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import freeze_support
from numba import jit

# Definição do problema

In [31]:
aulas: list[str] = [
    'M', # Matemática
    'L', # Língua Portuguesa
    'H', # História
    'C', # Ciências
    'R', # Ensino Religioso
    'I', # Inglês
    'F', # Educação Física
    'A', # Arte
]

In [32]:
case: str = 'ILLMMMCHIHLLMHHILLILHLMAHIAHLFHIMHLCILMCMLIMCMAAAILHLMCLIMLAIHCHMMLLLIMLIHHMLALMIMMAMLIRMFRMLILMMHIRLAIMALRACLMICMAHIFFM'

# Funções auxiliares

In [51]:
@jit
def separar_turmas(horario: str, quantidade_turmas: int = 4) -> list[str]:
    """ Separa o horário em turmas
    
    Args:
        horario (str): Horário das aulas
        quantidade_turmas (int, optional): Quantidade de turmas. Defaults to 4.
    
    Returns:
        list[str]: Lista de horários separados por turma
    """
    
    # Cria uma lista de strings vazias
    turmas: list[str] = ['' for _ in range(quantidade_turmas)]
    
    # Itera sobre o horário
    for i, aula in enumerate(horario):
        # Adiciona a aula na turma correspondente
        turmas[i % quantidade_turmas] += aula
    
    return turmas

separar_turmas(case)

['IMIMLHHLMIMCALIIMLILIMMLMLACCI',
 'LMHHLLIFHLLMIMMHMIHAMLFIHALLMF',
 'LCLHIMAHLMIALCLCLMHLMIRLIIRMAF',
 'MHLILAHICCMAHLAHLLMMARMMRMAIHM']

In [52]:
@jit
def separar_dias(turma: str) -> list[list[str]]:
    """ Separa a turma em dias
    
    Args:
        turma (str): Horário de uma turma
    
    Returns:
        list[list[str]]: Lista de horários separados por dia
    """
    
    # Cria uma lista de strings vazias
    dias: list[list[str]] = [['' for _ in range(6)] for _ in range(5)]
    
    # Itera sobre o horário
    for i, aula in enumerate(turma):
        # Adiciona a aula no dia correspondente
        dias[i // 6][i % 6] = aula
    
    return dias

separar_dias(separar_turmas(case)[0])

[['I', 'M', 'I', 'M', 'L', 'H'],
 ['H', 'L', 'M', 'I', 'M', 'C'],
 ['A', 'L', 'I', 'I', 'M', 'L'],
 ['I', 'L', 'I', 'M', 'M', 'L'],
 ['M', 'L', 'A', 'C', 'C', 'I']]

# Avaliação

In [35]:
requisitos: dict = {
    'M': 28,
    'L': 28,
    'H': 16,
    'C': 8,
    'R': 4,
    'I': 20,
    'F': 4
}

def quantidade_aulas(horario: str, requisitos: dict) -> int:
    """ Verifica se o horário tem a quantidade de aulas necessárias 
    
    Args:
        horario (str): Horário das aulas
        requisitos (dict): Requisitos de aulas por matéria
    
    Returns:
        int: Quantidade de aulas faltantes ou excedentes
    """
    
    # Conta a quantidade de aulas de cada matéria no horário especificado
    c: Counter = Counter(horario)
    # e.g. {'M': 28, 'L': 28, 'H': 16, 'C': 8, 'R': 4, 'I': 20, 'F': 4}
    
    n: int =  sum( # Somatório
        abs( # Valor absoluto para obter a diferença entre a quantidade de aulas esperada e a quantidade de aulas no horário
            requisitos[materia] - c.get(materia, 0) # Quantidade esperada daquela matéria - Quantidade de aulas daquela matéria no horário
        ) \
            for materia in requisitos # Aplica a operação para cada matéria, e soma o resultado
    )
    
    return n

quantidade_aulas(case, requisitos)

0

In [56]:
@jit
def validar_restricoes(horario: str) -> int:
    n: int = 0
    for turma in separar_turmas(horario):
        for d, dia in enumerate(separar_dias(turma)):
            
            # Verifica se o horário da aula de inglês é após as 10:50
            if 'I' in dia and dia.index('I') >= 4:
                n += 1
                
            if 'I' not in dia:
                n += 1
            
            # Verifica se o horário de Geo/História é após as 10:50
            if 'H' in dia and dia.index('H') >= 4:
                n += 1
                
            # Verifica se o horário de Educação Física é antes das 9:10
            if 'F' in dia and dia.index('F') < 2:
                n += 1
                
            # Verifica se o dia da educação física não é sexta-feira
            if 'F' in dia and d < 4:
                n += 1

    return n

validar_restricoes(case)

14

In [36]:
# Dicionário de aulas com seus respectivos caracteres
aula_dict = {
    'M': 'Matemática',
    'L': 'Língua Portuguesa',
    'H': 'História',
    'C': 'Ciências',
    'R': 'Ensino Religioso',
    'I': 'Inglês',
    'F': 'Educação Física',
    'A': 'Arte',
}

# Dicionário de restrições - AKA Limitação dos horários dos professores
restricoes = {
    0: {  # Segunda-feira
        'I': ['manhã'],
        'H': ['manhã'],
        'F': []  
    },

    1: {  # Terça-feira
        'I': ['manhã'],
        'H': ['manhã'],
        'F': []
    },

    2: {  # Quarta-feira
        'I': ['manhã'],
        'H': ['manhã'],  
        'F': []
    },

    3: {  # Quinta-feira
        'I': ['manhã'],
        'H': ['manhã'],
        'F': []
    },

    4: {  # Sexta-feira
        'I': ['manhã'],
        'H': ['manhã'],
        'F': [2,3,4,5]  # Educação Física deve estar na terceira até sexta aula (índice 1)
    }
}

In [37]:
t = separar_turmas(case)[0]
t

'IMIMLHHLMIMCALIIMLILIMMLMLACCI'

9

In [39]:
def validar_horario(horario, restricoes):
    pontos = 0
    for turma in range(4):
        offset = turma * 30
        for dia in range(5):
            aulas_do_dia = horario[offset + dia*6 : offset + (dia+1)*6]
            regras = restricoes[dia]
            for aula, condicoes in regras.items():
                if isinstance(condicoes, list):
                    for condicao in condicoes:
                        if isinstance(condicao, int):
                            if aulas_do_dia[condicao] != aula:
                                pontos += 1
                        elif isinstance(condicao, list):
                            if not any(aulas_do_dia[i] == aula for i in condicao):
                                pontos += 1
                        elif condicao == 'manhã' and aula in aulas_do_dia[4:]:
                            pontos += 1
                        elif condicao == 'somente' and aula not in aulas_do_dia:
                            pontos += 1
            count_ingles = aulas_do_dia.count('I')
            if count_ingles == 0:
                pontos += 1
            elif count_ingles > 1:
                pontos += (count_ingles - 1)
    return pontos

In [40]:
def verificar_sobreposicao(horario):
    pontuacao = 0
    sub_horarios = [horario[i:i+30] for i in range(0, len(horario), 30)]
    for dia in range(5):
        for aula_index in range(6):
            aulas_no_horario = [sub_horario[dia * 6 + aula_index] for sub_horario in sub_horarios]
            contador_aulas = {}
            for aula in aulas_no_horario:
                contador_aulas[aula] = contador_aulas.get(aula, 0) + 1
            for count in contador_aulas.values():
                if count > 1:
                    pontuacao += (count - 1)
    return pontuacao

In [41]:
def fitting(horario):
    contador = Counter(horario)
    pontuacao = sum(abs(requisitos[aula] - contador.get(aula, 0)) for aula in requisitos)
    pontuacao += validar_horario(horario, restricoes)
    pontuacao += verificar_sobreposicao(horario)  
    return pontuacao

In [42]:
fitting(case)

44